In [0]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, roc_curve, auc
from sklearn.svm import SVC
import spacy
import re
import string

In [2]:
!pip install unidecode

     |████████████████████████████████| 245kB 8.2MB/s 


In [0]:
from unidecode import unidecode

In [0]:
#data = pd.read_csv('final.csv')
data = pd.read_csv('/content/drive/My Drive/BERT/CSV_FINAL_EQUALIZADO.csv')
data = data.drop('Unnamed: 0', axis=1)
#data['review_rate'] = data['review_rate'].fillna(' ')
#data = data.fillna(data.mean())
data = data.drop_duplicates(subset=['review_body'])

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [7]:
data['review_rate'].value_counts()

50    11580
30    10558
40     4847
20     3335
10     2534
Name: review_rate, dtype: int64

In [0]:
def DATA_BINARIZADOR_RESULTADOS(data):
    RESULTADO_BINARIO = []

    for item in data['review_rate']:
        if item < 40:
            RESULTADO_BINARIO.append(0)
        elif item >= 40:
            RESULTADO_BINARIO.append(1)
        else:
            print("Deu feijoada aqui hein")

    data['RESULTADO_BINARIO'] = RESULTADO_BINARIO
    return data

In [9]:

data = DATA_BINARIZADOR_RESULTADOS(data)
data.head()

,review_rate,review_body,RESULTADO_BINARIO
0,50,Excelente experiência !! Atendimento impecável...,1
1,30,"Local agradável, porém, já fui em melhores, co...",0
2,50,Esta é a “milésima” vez que venho a este resta...,1
3,40,"O local, o atendimento e a comida, sensacionai...",1
4,50,"Comida espetacular, atendimento ímpar! Frank e...",1


In [0]:
def clear_text(text):
    #remove pontuacao, palavras com numeros, deixa o texto em caixa baixa e remove o texto entre colchetes
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = unidecode(text)
    return text

In [0]:
clear = lambda x: clear_text(x)
data_clean = pd.DataFrame(data.review_body.apply(clear))
#data_clean['review_rate'] = data['review_rate'].values
data_clean['RESULTADO_BINARIO'] = data['RESULTADO_BINARIO'].values


In [0]:
X_train, X_test, y_train, y_test = train_test_split(data_clean[['review_body']], data_clean['RESULTADO_BINARIO'], test_size=0.20, random_state=40)

In [0]:
t_vector = TfidfVectorizer()
t_vector.fit(data_clean['review_body'])
train_X = t_vector.transform(X_train['review_body'])
test_X = t_vector.transform(X_test['review_body'])


In [0]:
def validar(model, model_name, y, pred):
    print("Os escores do ", model_name," são:")
    print(' acc = ', accuracy_score(y, pred), ("\n"),
      'prec = ', precision_score(y, pred), ("\n"),
      'recall = ', recall_score(y, pred), ("\n"),
      'f1 = ', f1_score(y, pred))

    fpr, tpr, threshold = roc_curve(y, pred)
    roc_auc = auc(fpr, tpr)
    print(' auc = ', roc_auc)
    return True

<h1>Modelos</h1>

<h2> Regressão Logística </h2>

<h2>Treino</h2>

In [0]:
log = LogisticRegression()
log.fit(train_X, y_train)
log_pred = log.predict(train_X)


print(' acc = ', accuracy_score(y_train, log_pred), ("\n"),
      'prec = ', precision_score(y_train, log_pred), ("\n"),
      'recall = ', recall_score(y_train, log_pred), ("\n"),
      'f1 = ', f1_score(y_train, log_pred))

fpr, tpr, threshold = roc_curve(y_train, log_pred)
roc_auc = auc(fpr, tpr)
print(' auc = ', roc_auc)


 acc =  0.9125290111478903 
 prec =  0.916973886328725 
 recall =  0.9074257049479365 
 f1 =  0.912174809947664
 auc =  0.912535037450351


<h2>Teste</h2>

In [0]:
log_pred_test = log.predict(test_X)

print(' acc = ', accuracy_score(y_test, log_pred_test), ("\n"),
      'prec = ', precision_score(y_test, log_pred_test), ("\n"),
      'recall = ', recall_score(y_test, log_pred_test), ("\n"),
      'f1 = ', f1_score(y_test, log_pred_test))

fpr, tpr, threshold = roc_curve(y_test, log_pred_test)
roc_auc = auc(fpr, tpr)
print(' auc = ', roc_auc)


 acc =  0.8828184446811749 
 prec =  0.8874767513949163 
 recall =  0.8755351681957186 
 f1 =  0.8814655172413793
 auc =  0.8827842457155509


<h2>Random Forest otimizando default</h2>

In [0]:
def random_grid(x, labels):
    param_grid = [
    {'n_estimators': [3, 10, 30], 'max_features': [2,4,6,8]}, 
    {'bootstrap': [False], 'n_estimators': [3,10], 'max_features': [2, 3, 4]},
    ]   
    grid_search = GridSearchCV(RandomForestClassifier(), param_grid, cv=5)
    grid_search.fit(x, labels)
    print(grid_search.best_estimator_)
    return grid_search.best_estimator_

model = random_grid(train_X, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features=8,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=30,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)


<h2>Treino</h2>

In [0]:
rf_pred = model.predict(train_X)
print(' acc = ', accuracy_score(y_train, rf_pred), ("\n"),
      'prec = ', precision_score(y_train, rf_pred), ("\n"),
      'recall = ', recall_score(y_train, rf_pred), ("\n"),
      'f1 = ', f1_score(y_train, rf_pred))

fpr, tpr, threshold = roc_curve(y_train, rf_pred)
roc_auc = auc(fpr, tpr)
print(' auc = ', roc_auc)

 acc =  0.9998478103717232 
 prec =  0.9999239832763208 
 recall =  0.9997719844949456 
 f1 =  0.9998479781088476
 auc =  0.9998478999116508


<h2>Teste</h2>

In [0]:
rf_pred_test = model.predict(test_X)
print(' acc = ', round(accuracy_score(y_test, rf_pred_test), 5), ("\n"),
      'prec = ', round(precision_score(y_test, rf_pred_test), 5), ("\n"),
      'recall = ', round(recall_score(y_test, rf_pred_test), 5), ("\n"),
      'f1 = ', round(f1_score(y_test, rf_pred_test), 5))

fpr, tpr, threshold = roc_curve(y_test, rf_pred_test)
roc_auc = auc(fpr, tpr)
print(' auc = ', round(roc_auc, 5))

 acc =  0.81373 
 prec =  0.80888 
 recall =  0.81927 
 f1 =  0.81404
 auc =  0.81375


<h2>Random Forest: grid seach otimizando acurácia</h2>

In [0]:
def random_grid(x, labels):
    param_grid = [
    {'n_estimators': [3, 10, 30], 'max_features': [2,4,6,8]}, 
    {'bootstrap': [False], 'n_estimators': [3,10], 'max_features': [2, 3, 4]},
    ]   
    grid_search = GridSearchCV(RandomForestClassifier(), param_grid, cv=5, scoring='accuracy')
    grid_search.fit(x, labels)
    print(grid_search.best_estimator_)
    return grid_search.best_estimator_

model = random_grid(train_X, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features=4,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=30,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)


<h2>Treino</h2>

In [0]:
rf_pred = model.predict(train_X)
print(' acc = ', accuracy_score(y_train, rf_pred), ("\n"),
      'prec = ', precision_score(y_train, rf_pred), ("\n"),
      'recall = ', recall_score(y_train, rf_pred), ("\n"),
      'f1 = ', f1_score(y_train, rf_pred))

fpr, tpr, threshold = roc_curve(y_train, rf_pred)
roc_auc = auc(fpr, tpr)
print(' auc = ', roc_auc)

 acc =  0.9998858577787924 
 prec =  1.0 
 recall =  0.9997719844949456 
 f1 =  0.9998859792482232
 auc =  0.9998859922474728


<h2>Teste</h2>

In [0]:
rf_pred_test = model.predict(test_X)
print(' acc = ', round(accuracy_score(y_test, rf_pred_test), 5), ("\n"),
      'prec = ', round(precision_score(y_test, rf_pred_test), 5), ("\n"),
      'recall = ', round(recall_score(y_test, rf_pred_test), 5), ("\n"),
      'f1 = ', round(f1_score(y_test, rf_pred_test), 5))

fpr, tpr, threshold = roc_curve(y_test, rf_pred_test)
roc_auc = auc(fpr, tpr)
print(' auc = ', round(roc_auc, 5))

 acc =  0.81236 
 prec =  0.81601 
 recall =  0.80428 
 f1 =  0.8101
 auc =  0.81232


<h2>Random Forest otimizando AUC </h2>

In [0]:
def random_grid(x, labels):
    param_grid = [
    {'n_estimators': [3, 10, 30], 'max_features': [2,4,6,8]}, 
    {'bootstrap': [False], 'n_estimators': [3,10], 'max_features': [2, 3, 4]},
    ]   
    grid_search = GridSearchCV(RandomForestClassifier(), param_grid, cv=5, scoring='roc_auc')
    grid_search.fit(x, labels)
    print(grid_search.best_estimator_)
    return grid_search.best_estimator_

model = random_grid(train_X, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features=8,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=30,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)


<h2>Treino</h2>

In [0]:
rf_pred = model.predict(train_X)
print(' acc = ', accuracy_score(y_train, rf_pred), ("\n"),
      'prec = ', precision_score(y_train, rf_pred), ("\n"),
      'recall = ', recall_score(y_train, rf_pred), ("\n"),
      'f1 = ', f1_score(y_train, rf_pred))

fpr, tpr, threshold = roc_curve(y_train, rf_pred)
roc_auc = auc(fpr, tpr)
print(' auc = ', roc_auc)

 acc =  0.999809762964654 
 prec =  0.9997720018239854 
 recall =  0.9998479896632971 
 f1 =  0.999809994299829
 auc =  0.9998097178241825


<h2>Teste</h2>

In [0]:
rf_pred_test = model.predict(test_X)
print(' acc = ', round(accuracy_score(y_test, rf_pred_test), 5), ("\n"),
      'prec = ', round(precision_score(y_test, rf_pred_test), 5), ("\n"),
      'recall = ', round(recall_score(y_test, rf_pred_test), 5), ("\n"),
      'f1 = ', round(f1_score(y_test, rf_pred_test), 5))

fpr, tpr, threshold = roc_curve(y_test, rf_pred_test)
roc_auc = auc(fpr, tpr)
print(' auc = ', round(roc_auc, 5))

 acc =  0.81205 
 prec =  0.81747 
 recall =  0.80122 
 f1 =  0.80927
 auc =  0.812


<h2>SVM otimizando default</h2>

In [0]:
def svm_grid(x, labels):
    param_grid = [
    {'kernel': ['rbf', 'sigmoid'],
        'C': [0.1, 1.0, 10.0, 100.0],
        'degree': [1, 2, 3, 4]}
    ]   
    grid_search = GridSearchCV(SVC(), param_grid, cv=5)
    grid_search.fit(x, labels)
    print(grid_search.best_estimator_)
    return grid_search.best_estimator_

model_svc = svm_grid(train_X, y_train)

<h2>Treino</h2>

In [0]:
svm_pred = model.predict(train_X)
print(' acc = ', round(accuracy_score(y_train, svm_pred), 5), ("\n"),
      'prec = ', round(precision_score(y_train, svm_pred), 5), ("\n"),
      'recall = ', round(recall_score(y_train, svm_pred), 5), ("\n"),
      'f1 = ', round(f1_score(y_train, svm_pred), 5))

fpr, tpr, threshold = roc_curve(y_train, svm_pred)
roc_auc = auc(fpr, tpr)
print(' auc = ', round(roc_auc, 5))

<h2>Teste</h2>

In [0]:
svm_pred_test = model.predict(test_X)
print(' acc = ', round(accuracy_score(y_test, svm_pred_test), 5), ("\n"),
      'prec = ', round(precision_score(y_test, svm_pred_test), 5), ("\n"),
      'recall = ', round(recall_score(y_test, svm_pred_test), 5), ("\n"),
      'f1 = ', round(f1_score(y_test, svm_pred_test), 5))

fpr, tpr, threshold = roc_curve(y_test, svm_pred_test)
roc_auc = auc(fpr, tpr)
print(' auc = ', round(roc_auc, 5))

<h2>SVM otimizando acurácia</h2>

In [0]:
def svm_grid(x, labels):
    param_grid = [
    {'kernel': ['rbf', 'sigmoid'],
        'C': [0.1, 1.0, 10.0, 100.0],
        'degree': [1, 2, 3]}
    ]   
    grid_search = GridSearchCV(SVC(), param_grid, cv=5, scoring='accuracy')
    grid_search.fit(x, labels)
    print(grid_search.best_estimator_)
    return grid_search.best_estimator_

model_svc = svm_grid(train_X, y_train)

In [0]:
SVC().get_params().keys() #os parametros que dá pra alterar

<h2>Treino</h2>

In [0]:
svm_pred = model.predict(train_X)
print(' acc = ', round(accuracy_score(y_train, svm_pred), 5), ("\n"),
      'prec = ', round(precision_score(y_train, svm_pred), 5), ("\n"),
      'recall = ', round(recall_score(y_train, svm_pred), 5), ("\n"),
      'f1 = ', round(f1_score(y_train, svm_pred), 5))

fpr, tpr, threshold = roc_curve(y_train, svm_pred)
roc_auc = auc(fpr, tpr)
print(' auc = ', round(roc_auc, 5))

<h2>Teste</h2>

In [0]:
svm_pred_test = model.predict(test_X)
print(' acc = ', round(accuracy_score(y_test, svm_pred_test), 5), ("\n"),
      'prec = ', round(precision_score(y_test, svm_pred_test), 5), ("\n"),
      'recall = ', round(recall_score(y_test, svm_pred_test), 5), ("\n"),
      'f1 = ', round(f1_score(y_test, svm_pred_test), 5))

fpr, tpr, threshold = roc_curve(y_test, svm_pred_test)
roc_auc = auc(fpr, tpr)
print(' auc = ', round(roc_auc, 5))

In [0]:
#refazendo os grid search, optando por otimizar a auc

<h2>SVM otimizando AUC</h2>

In [0]:
def svm_grid(x, labels):
    param_grid = [
    {'kernel': ['rbf', 'sigmoid'],
        'C': [0.1, 1.0, 10.0, 100.0],
        'degree': [1, 2, 3, 4]}
    ]   
    grid_search = GridSearchCV(SVC(), param_grid, cv=5, scoring='roc_auc')
    grid_search.fit(x, labels)
    print(grid_search.best_estimator_)
    return grid_search.best_estimator_

model_svc = svm_grid(train_X, y_train)

<h2>Treino</h2>

In [0]:
svm_pred = model.predict(train_X)
print(' acc = ', round(accuracy_score(y_train, svm_pred), 5), ("\n"),
      'prec = ', round(precision_score(y_train, svm_pred), 5), ("\n"),
      'recall = ', round(recall_score(y_train, svm_pred), 5), ("\n"),
      'f1 = ', round(f1_score(y_train, svm_pred), 5))

fpr, tpr, threshold = roc_curve(y_train, svm_pred)
roc_auc = auc(fpr, tpr)
print(' auc = ', round(roc_auc, 5))

<h2>Teste</h2>

In [0]:
svm_pred_test = model.predict(test_X)
print(' acc = ', round(accuracy_score(y_test, svm_pred_test), 5), ("\n"),
      'prec = ', round(precision_score(y_test, svm_pred_test), 5), ("\n"),
      'recall = ', round(recall_score(y_test, svm_pred_test), 5), ("\n"),
      'f1 = ', round(f1_score(y_test, svm_pred_test), 5))

fpr, tpr, threshold = roc_curve(y_test, svm_pred_test)
roc_auc = auc(fpr, tpr)
print(' auc = ', round(roc_auc, 5))